In [3]:

import metrics_calc
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


Using device: cpu


In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load GPT-2 pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)




GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:

import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 Model & Tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Read validation dataset (Excel file)
input_excel = r"C:\קבצים אחרונים לגיבוי\NLP-Final-Project---Dreams-Interpreter\DREAMS DATA\dreams_and_interpretations_Freud.xlsx"
df = pd.read_excel(input_excel)

# Ensure the dataset has a 'Dream' column
if "Dream" not in df.columns:
    raise ValueError("CSV file must contain a 'Dream' column.")

# Function to generate interpretations
def generate_interpretation(dream):
    # Improved Few-Shot Learning Prompt
    prompt = (
        "You are an expert dream interpreter. Based on historical research and psychological theories, provide meaningful interpretations for given dream symbols.\n\n"
        "Dream: Flying\nInterpretation: This dream symbolizes a deep desire for freedom, independence, and escaping from limitations. It often reflects confidence, personal growth, or an urge to break free from restrictions.\n\n"
        "Dream: Teeth falling out\nInterpretation: This dream commonly represents anxiety, fear of aging, or concerns about personal power and appearance. It can also be linked to major life changes or feelings of vulnerability.\n\n"
        "Dream: Being chased\nInterpretation: A classic stress-related dream, often reflecting avoidance of real-life conflicts, fears, or responsibilities. It may indicate feelings of insecurity or repressed trauma.\n\n"
        "Dream: Standing in an empty room\nInterpretation: Suggests feelings of loneliness, self-reflection, or awaiting new opportunities. The atmosphere of the room (light, dark, chaotic) adds deeper meaning to the dream.\n\n"
        f"Dream: {dream}\nInterpretation:"
    )
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate interpretation
    with torch.no_grad():
        output = model.generate(
            input_ids, 
            max_length=500,  # Increased length for more detailed responses
            num_return_sequences=1,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )
    
    interpretation = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Extract and clean output
    return interpretation.split("Interpretation:")[-1].strip()

# Generate interpretations
df["Generated_Interpretation"] = df["Dream"].apply(generate_interpretation)

# Save to new CSV
output_csv = r"C:\קבצים אחרונים לגיבוי\NLP-Final-Project---Dreams-Interpreter\Barak model\GPT2 Model\output gpt2\dreams_with_generated_interpretations_gpt2.csv"
df.to_csv(output_csv, index=False)

print(f"Generated interpretations saved to {output_csv}")



Generated interpretations saved to C:\קבצים אחרונים לגיבוי\NLP-Final-Project---Dreams-Interpreter\Barak model\GPT2 Model\output gpt2\dreams_with_generated_interpretations_gpt2.csv


In [8]:

output_metrics_csv = "C:/קבצים אחרונים לגיבוי/NLP-Final-Project---Dreams-Interpreter\Barak model/GPT2 Model/output gpt2/FSL_gpt2_metrics_results.csv"
metrics_calc.evaluate_text_metrics(tokenizer, model, output_csv, output_metrics_csv)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

PermissionError: [Errno 13] Permission denied: 'C:/קבצים אחרונים לגיבוי/NLP-Final-Project---Dreams-Interpreter\\Barak model/GPT2 Model/output gpt2/FSL_gpt2_metrics_results.csv'

ModuleNotFoundError: No module named 'exceptions'